# Imports and Setup

In [5]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from scipy import ndimage
import pickle
import copy
import random
import time

torch.set_printoptions(precision=3)
cuda = True if torch.cuda.is_available() else False
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
datasetCifar = True
amensiac = False
#[0, 1, len(dataLoad) * 0.1, len(dataLoad) * 0.2, len(dataLoad) * 0.3, len(dataLoad) * 0.4, len(dataLoad) * (0.5), len(dataLoad) * (0.6), len(dataLoad) * (0.7), len(dataLoad) * (0.8), len(dataLoad) * (0.9)]
idx = 4


# Data Entry and Processing

In [2]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,),(0.5)),
                                ])

In [3]:
# Using CIFAR100
traindata = datasets.CIFAR100('./data', download=True, train=True, transform=transform)
testdata = datasets.CIFAR100('./data', download=True, train=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Loaders that give 64 example batches
cifar_train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
cifar_test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)

In [5]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,)),
                                ])

In [6]:
# Using MNIST
traindata = datasets.MNIST('./data', download=True, train=True, transform=transform)
testdata = datasets.MNIST('./data', download=True, train=False, transform=transform)

In [7]:
mnist_train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
mnist_test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)


In [8]:
data, label = next(iter(cifar_train_loader))
print(data.size())
print(len(cifar_train_loader.dataset))
print(len(cifar_train_loader))
# for batch_idx, (data, target) in enumerate(mnist_train_loader):
#     print(batch_idx) #938
# print(mnist_train_loader)

torch.Size([64, 3, 32, 32])
50000
782


# Model

In [9]:
# Hyperparameters
log_interval = 10
num_classes = 100 if datasetCifar else 10
torch.backends.cudnn.enabled = True
criterion = F.nll_loss

In [10]:
# load resnet 18 and change to fit problem dimensionality
resnet = models.resnet18()
resnet.conv1 = nn.Conv2d(3, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False) if datasetCifar else nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
resnet.fc = nn.Sequential(nn.Linear(512, num_classes), nn.LogSoftmax(dim=1))
optimizer = optim.Adam(resnet.parameters())
# optimizer.to(device)
resnet = resnet.to(device)
print(resnet)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# Training method that saves batch updates
def train(model, epoch, loader, returnable=False, batchStop=0):
  model.train()
  deltas = []
  if amensiac:
    for _ in range(50):
        delta = {}
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
              delta[param_tensor] = 0
        deltas.append(delta)
    before = {}
    for param_tensor in model.state_dict():
        if "weight" in param_tensor or "bias" in param_tensor:
            before[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
  for batch_idx, (data, target) in enumerate(loader):
    if batch_idx <= len((loader)) - 1 - batchStop:
      optimizer.zero_grad()
      output = model(data.to(device))
      loss = criterion(output, target.to(device))
      loss.backward()
      optimizer.step()
      if batch_idx % 10 == 0 and batch_idx < 500 and amensiac:
        after = {}
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
            after[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
        for key in before:
          deltas[batch_idx // 10][key] = after[key] - before[key]
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
            before[param_tensor] = model.state_dict()[param_tensor].detach().cpu()
      if batch_idx % log_interval == 0:
        print("\rEpoch: {} [{:6d}]\tLoss: {:.6f}".format(
            epoch, batch_idx*len(data),  loss.item()
        ), end="")
    else:
      break
  return deltas

In [12]:
# Testing method
def test(model, loader, dname="Test set", printable=True):
  model.eval()
  test_loss = 0
  total = 0
  correct = 0
  with torch.no_grad():
    for data, target in loader:
      output = model(data.to(device))
      total += target.size()[0]
      test_loss += criterion(output, target.to(device)).item()
      _, pred = torch.topk(output, 1, dim=1, largest=True, sorted=True)
      for i, t in enumerate(target):
        if t in pred[i]:
            correct += 1
  test_loss /= len(loader.dataset)
  if printable:
    print('{}: Mean loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        dname, test_loss, correct, total, 
        100. * correct / total
        ))
  return 1. * correct / total

# Original Training

In [13]:
trainingepochs = 10
num_classes = 100 if datasetCifar else 10
counter = 0
for param_tensor in resnet.state_dict():
    if "weight" in param_tensor or "bias" in param_tensor:
        counter+=1
print(counter)
dataLoad = cifar_train_loader if datasetCifar else mnist_train_loader
dataTest = cifar_test_loader if datasetCifar else mnist_test_loader
stop = [0, 1, len(dataLoad) * 0.1, len(dataLoad) * 0.2, len(dataLoad) * 0.3, len(dataLoad) * 0.4, len(dataLoad) * (0.5), len(dataLoad) * (0.6), len(dataLoad) * (0.7), len(dataLoad) * (0.8), len(dataLoad) * (0.9)]
batchStop = stop[idx]


62


In [14]:
# Train new model for n epochs, saving parameter updates for
# sensitive batches

trainstart = time.process_time()
deltas = []
trainAcc = 0
if amensiac:  
  for _ in range(50):
    delta = {}
    for param_tensor in resnet.state_dict():
      if "weight" in param_tensor or "bias" in param_tensor:
          delta[param_tensor] = 0
    deltas.append(delta)
    
for epoch in range(1, trainingepochs+1):
  starttime = time.process_time()
  # train(resnet, epoch, all_data_train_loader, returnable=False)
  # batch = train(resnet, epoch, mnist_train_loader, returnable=True) 
  batch = train(resnet, epoch, dataLoad, returnable=True, batchStop=batchStop)
  if amensiac:
    for i in range(50):
      for key in deltas[i]:
          deltas[i][key] = batch[i][key] + deltas[i][key]
  # test(resnet, mnist_test_loader, dname="All data")
  trainAcc = test(resnet, dataTest, dname="All data")
  if epoch == trainingepochs:
    print("Accuracy: ", trainAcc)

  print(f"Time taken: {time.process_time() - starttime}")
endTimeTrain = time.process_time() - trainstart
print(f"Time taken for training: {endTimeTrain}")
accTime = {"accuracy": trainAcc, "time": endTimeTrain}


Epoch: 1 [ 23680]	Loss: 0.166844All data: Mean loss: 0.0020, Accuracy: 9616/10000 (96%)
Time taken: 7.702917798
Epoch: 2 [ 23680]	Loss: 0.032168All data: Mean loss: 0.0016, Accuracy: 9685/10000 (97%)
Time taken: 7.430698835000001
Epoch: 3 [ 23680]	Loss: 0.061885All data: Mean loss: 0.0011, Accuracy: 9792/10000 (98%)
Time taken: 7.146857168
Epoch: 4 [ 23680]	Loss: 0.153832All data: Mean loss: 0.0009, Accuracy: 9841/10000 (98%)
Time taken: 7.929298318000001
Epoch: 5 [ 23680]	Loss: 0.024110All data: Mean loss: 0.0008, Accuracy: 9824/10000 (98%)
Time taken: 7.542429286999997
Epoch: 6 [ 23680]	Loss: 0.003942All data: Mean loss: 0.0007, Accuracy: 9872/10000 (99%)
Time taken: 7.965778234999995
Epoch: 7 [ 23680]	Loss: 0.106235All data: Mean loss: 0.0009, Accuracy: 9837/10000 (98%)
Time taken: 7.552382590000001
Epoch: 8 [ 23680]	Loss: 0.000542All data: Mean loss: 0.0005, Accuracy: 9892/10000 (99%)
Time taken: 7.512394193999995
Epoch: 9 [ 23680]	Loss: 0.052652All data: Mean loss: 0.0009, Accurac

In [15]:
if batchStop != 0:
    accAndTime = F"CIFARBatchStop" + str(batchStop) + ".pickle" if datasetCifar else F"MNISTBatchStop" + str(batchStop) + ".pickle"
    with open(accAndTime, 'wb') as handle:
        pickle.dump(accTime, handle)

In [16]:
if batchStop != 0:
    with open(accAndTime, 'rb') as handle:
        b = pickle.load(handle)
    print(accAndTime)
    print(b)

MNISTBatchStop562.8.pickle
{'accuracy': 0.9889, 'time': 75.906329421}


In [15]:
deltaPickles = F"CIFARdeltaPickles.pickle" if datasetCifar else F"Mnistdeltapickles.pickle"

with open(deltaPickles, 'wb') as handle:
    pickle.dump(deltas, handle)

In [16]:
# # Train new model for n epochs, saving parameter updates for
# # ignoring 
# deltas = []
# for _ in range(50):
#   delta = {}
#   for param_tensor in resnet.state_dict():
#     if "weight" in param_tensor or "bias" in param_tensor:
#         delta[param_tensor] = 0
#   deltas.append(delta)
# for epoch in range(1, trainingepochs+1):
#   starttime = time.process_time()
#   # train(resnet, epoch, all_data_train_loader, returnable=False)
#   batch = train(resnet, epoch, mnist_train_loader, returnable=True) 
#   for i in range(50):
#     for key in deltas[i]:
#         deltas[i][key] = batch[i][key] + deltas[i][key]
#   test(resnet, mnist_test_loader, dname="All data")
#   print(f"Time taken: {time.process_time() - starttime}")

In [17]:
# path = F"resnet/selective_mnist.pt"
path = F"resnet/selective_CIFAR.pt" if datasetCifar else F"resnet/selective_mnist.pt"
torch.save({
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, path)

In [18]:
# path = F"resnet/selective_CIFAR.pt" if datasetCifar else F"resnet/selective_mnist.pt"
checkpoint = torch.load(path)
resnet.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [19]:
accuracy = [[],[],[],[],[],[],[],[],[],[]]
recordTime = [[],[],[],[],[],[],[],[],[],[]]
print(type(deltas))
# print(deltas)
# fileo = open(path, 'rb')
# deltas = pickle.load(fileo)
# fileo.close()
b = []
with open(deltaPickles, 'rb') as handle:
    b = pickle.load(handle)
print(len(b))
print(len(deltas))
import os
print(os.path.getsize(deltaPickles)/1073741824, ' GB')
# import sys
# print(sys.getsizeof(deltas)/1073741824)


<class 'list'>
50
50
2.0824923319742084  GB


In [20]:
# Begin amnesiac unlearning process, evaluating
# model accuracy as batches are removed
# 1 iteration = 1 batch
for j in range(10):
    random.shuffle(deltas)
    resnet.load_state_dict(checkpoint['model_state_dict'])
    for i in range(50):
        unlearnItStart = time.process_time()
        print(f"\riteration {j},{i}", end="")
        const = 1
        with torch.no_grad():
            state = resnet.state_dict()
            for param_tensor in state:
                if "weight" in param_tensor or "bias" in param_tensor:
                  state[param_tensor] = state[param_tensor] - const*deltas[i][param_tensor].cuda()
        resnet.load_state_dict(state)
        accuracy[j].append(test(resnet, dataTest, dname="All data"))
        end = time.process_time() - unlearnItStart
        recordTime[j].append(end)
        print(f"Time taken for unlearning: {end}")

iteration 0,0All data: Mean loss: 0.0017, Accuracy: 9663/10000 (97%)
Time taken for unlearning: 1.4576336670000103
iteration 0,1All data: Mean loss: 0.0017, Accuracy: 9692/10000 (97%)
Time taken for unlearning: 1.4356638539999835
iteration 0,2All data: Mean loss: 0.0089, Accuracy: 8679/10000 (87%)
Time taken for unlearning: 1.4359116960000051
iteration 0,3All data: Mean loss: 0.0125, Accuracy: 8100/10000 (81%)
Time taken for unlearning: 1.4364724149999972
iteration 0,4All data: Mean loss: 0.0109, Accuracy: 8208/10000 (82%)
Time taken for unlearning: 1.4349128039999925
iteration 0,5All data: Mean loss: 0.0108, Accuracy: 8186/10000 (82%)
Time taken for unlearning: 1.4313004409999905
iteration 0,6All data: Mean loss: 0.0095, Accuracy: 8265/10000 (83%)
Time taken for unlearning: 1.4373474740000063
iteration 0,7All data: Mean loss: 0.0081, Accuracy: 8362/10000 (84%)
Time taken for unlearning: 1.436412426000004
iteration 0,8All data: Mean loss: 0.0117, Accuracy: 7432/10000 (74%)
Time taken f

In [22]:
# path = F"selective_acc_mnist.pk"
# with open(path, 'w') as f:
#   for data in accuracy:
#     f.write(f"{data},")

# path = F"selective_time_mnist.pk"
# with open(path, 'w') as f:
#   for t in recordTime:
#     f.write(f"{t},")

In [21]:
accuracyStore = F"selective_acc_CIFAR.pk" if datasetCifar else F"selective_acc_mnist.pk"
f = open(accuracyStore, "wb")
pickle.dump(accuracy, f)
f.close()

In [22]:
timeStore = F"selective_time_CIFAR.pk" if datasetCifar else F"selective_time_mnist.pk"
f = open(timeStore, "wb")
pickle.dump(recordTime, f)
f.close()

In [6]:
accuracyStore = F"selective_acc_CIFAR.pk" if datasetCifar else F"selective_acc_mnist.pk"
timeStore = F"selective_time_CIFAR.pk" if datasetCifar else F"selective_time_mnist.pk"
res = []
with open(accuracyStore, 'rb') as handle:
    res = pickle.load(handle)
print((res[0][0], res[0][9], res[0][19], res[0][29], res[0][39], res[0][49]))
letime = []
with open(timeStore, 'rb') as handle:
    letime = pickle.load(handle)
print(letime[0][0], letime[0][9], letime[0][19], letime[0][29], letime[0][39], letime[0][49])
unlearnTime = letime[0][0] + letime[0][9] + letime[0][19] + letime[0][29] + letime[0][39] + letime[0][49]
print('unlearn time')

(0.2316, 0.015, 0.0106, 0.0126, 0.0104, 0.0108)
1.6085630569999978 1.6013245839999968 1.59915330299998 1.6008214839999937 1.5998080219999906 1.6055981650000035
